Antimony is imported as sb (symbol)
Tellurium is imported as te (symbol)
My program can be called: neodymium (nd) or nobelium (no) from node but very similar to numpy! Better tellurium model builder (telmb)

Better: My program will be called Molybdenum!!! Imported as mb which stands for Model Builder

## Source information

 - https://github.com/sys-bio/tellurium/blob/develop/tellurium/teconverters/convert_antimony.py
 - https://simplesbml.readthedocs.io/en/latest/_modules/simplesbml.html
 - https://github.com/sys-bio/simplesbml/blob/master/simplesbml/simplesbml.py
 - https://simplesbml.readthedocs.io/en/latest/

## Example representations.

- `con_data` is specific to the nodes, and I call it graphical representation.
- `mod_data` is actually the molybdenum model format.

In [88]:
con_data = {'nodes':  [{'id': 1, 'title': 'E', 'x': 228.1999969482422, 'y': 169.60000610351562, 'nodeClass': 'species'},
                       {'id': 2, 'title': 'S', 'x': 227.1999969482422, 'y': 323.6000061035156, 'nodeClass': 'species'},
                       {'id': 3, 'title': 'ES', 'x': 615.2000122070312, 'y': 226.60000610351562, 'nodeClass': 'species'},
                       {'id': 4, 'title': 'P', 'x': 959.2000122070312, 'y': 222.60000610351562, 'nodeClass': 'species'},
                       {'id': 5, 'title': 'R1', 'x': 400.20001220703125, 'y': 229.60000610351562, 'nodeClass': 'reaction'},
                       {'id': 6, 'title': 'R2', 'x': 776.2000122070312, 'y': 229.60000610351562, 'nodeClass': 'reaction'}],
             'edges': [{'source': 1, 'target': 5}, 
                       {'source': 2, 'target': 5},
                       {'source': 5, 'target': 3},
                       {'source': 3, 'target': 6},
                       {'source': 6, 'target': 4}]} 


## this was the original one
# mod_data = {'species': {'1': {'name': 'E', 'concentration': 10.0, 'fixed': False},
#                         '2': {'name': 'S', 'concentration': 10.0, 'fixed': False},
#                         '3': {'name': 'ES', 'concentration': 10.0, 'fixed': False},
#                         '4': {'name': 'P', 'concentration': 10.0, 'fixed': False}},
#             'reactions': {'5': {'name': 'R1', 'reagents': ['E', 'S'], 'products': ['ES'], 'reversible': False},
#                           '6': {'name': 'R2', 'reagents': ['ES'], 'products': ['P'], 'reversible': False}}}

## but I prefer this one
mod_data = {'species': {'spec1': {'name': 'E', 'concentration': 10.0, 'fixed': False}, # if name starts with $ will set as fixed
                        'spec2': {'name': 'S', 'concentration': 10.0, 'fixed': False},
                        'spec3': {'name': 'ES', 'concentration': 10.0, 'fixed': False},
                        'spec4': {'name': 'P', 'concentration': 10.0, 'fixed': False}},
            'reactions': {'reac5': {'name': 'R1', 'reagents': ['E', 'S'], 'products': ['ES'], 'reversible': False},
                          'reac6': {'name': 'R2', 'reagents': ['ES'], 'products': ['P'], 'reversible': False}},
            'params': {'param1': {'name': 'Kcat', 'value': 12}, #units in the future
                       'param2': {'name': 'Kmax', 'value': 10}},
            # optionally, can include simulation parameters and node_to_id values
            'node_to_id': {1: 'spec1',
                           2: 'spec2',
                           3: 'spec3',
                           4: 'spec4',
                           5: 'spec5'},
            'simpars': {'init': 0, 'fin': 10, 'steps': 200}
           }

How my module should be used

```python
## 1. Initialize the model

# import molybdenum as mb
# model = mb.ModelRepresentation()
# or
from molybenum import ModelRepresentation()
model = ModelRepresentation()

# this custom internal representation is known as molybdenum. Differential fact is that has an ID for each element,
# which allows for easy renaming of any parameter

model.species #empty now but initialized
model.reactions #empty now but initialized
model.events #support not yet implemented
model.simparams #simulation parameters

## 2. Fill in the information (species, reactions...)
# graph representation of nodes and edges
graph_rep = {}

# this function should have some checks defined inside the modelrepresentation class to ensure all IDs are unique
# note that update_from_nodes should only modify names and connections.
# If connection is modified reaction value or string should be modified
# But do not erase previous values of concentration if specie is not new or something.\
# Initialize with Nan, or as empty..? so that it can be displayed on form as missing
# IDs help keep track of changes in the name
model.update_from_graph(graph_rep)

model.species # no longer empty
model.reactions # no longer empty

# form is created by javascript directly from model.species or model.reactions
# form species and reactions are always listed in order of the ID of the species or reaction, so that 
# form values are passed in as a json or dictionary or something... 
# form does not alter the connections (reaction/products), only the others
# I guess reactions shall be dropdowns with table for parameters or something like this...
# when user types in new expression for a reaction, new parameters are created if they haven't been defined. Parameter's names can change in the parameter block and this will also update the expression (will have to implement this)
form_rep = {}
model.update_from_form(form_rep)

## 3. Export the model
model.tosimpleSbml() # I have to build this one (add species, etc...)
model.toSBML() # inside, calls tosimpleSbml and then the function to export to SBML string
model.toAntimony() # inside, calls toSBML to get SBML string and then convert this to antimony using tellurium function
model.toGraph() # go from the internal representation to the graphic representation

## 4. Run the model
sim_param = {} # parameters as you would set them in tellurium
model.update_sim_params(sim_param) # get the simulation parameters

model.run() # use tellurium to run the model

## 5. Alternative ways to initialize (they do not update, just initialize the model)
model.loadm(molybenummodel) # takes as input molybdenummodel, basically assigns species to species and reactions to reactions...
model.fromsimpleSbml(simpleSbmlmodel) # take SBML model and get the representation. Assign arbitrary IDs I guess... so that I can make node representation
model.fromAntimony(antimonymodel) # get SBML string from antimony, then use antimony to export to SBML and then use simpleSbml to go from SBML to simpleSBML and finally use fromsimpleSbml to get model inside
```

**Initial sketch**

In [80]:
class ModelRepresentation(object):
    def __init__(self):
        self.species = dict()
        self.reactions = dict()
        # not implemented yet
        # self.events = dict()
        # self.simparam = dict()

    def loadm(self, molybdenum_model):
        self.species = molybdenum_model['species']
        self.reactions = molybdenum_model['reactions']
    
    def tosimpleSbml(self):
        """Need to write this up now"""
        simpSbml_rep = None
        return simpSbml_rep
    def toSBMLstr(self):
        # gets smbl
        simpSbml_rep = self.tosimpleSbml()
        #toSBML is also a function from simpleSBML models that gets the sbml string, (confusing?)
        sbml_str = simpSbml_rep.toSBML()
        return sbml_str
    def tosimpleSbmlWriteup(self):
        simpSBML_rep = self.toSBMLstr()
        sbml_rep = simpSBML_rep.toSBML()
        sbml_writeup = simplesbml.simplesbml.writeCodeFromString(sbml_rep)
        return sbml_writeup

    def toAntimony(self):
        import tellurium as te
        r = te.antimonyConverter()
        sbml_str = self.toSBMLstr()
        sb_rep = r.sbmlToAntimony(sbml_str)[1]
        return sb_rep
    def toGraph(self):
        """Need to write this one up"""
        # for this, new X and Y values should be blank or zero even if looks awful, it is just for further development
        return None

## Some examples on how to convert simpleSBML to other formats

Start creating a simpleSBML model (will later be a simpleSBML model created from the Molybdenum model representation)

In [85]:
import simplesbml
model = simplesbml.SbmlModel()
model.addSpecies('E', 5e-21)
model.addSpecies('S', 1e-20)
model.addSpecies('ES', 0.0)
model.addSpecies('P', 0.0)
model.addParameter('koff', 0.2)
model.addParameter('kon', 1000000.0)
model.addParameter('kcat', 0.1)
model.addReaction(['E', 'S'], ['ES'], '(kon*E*S-koff*ES)', rxn_id='veq')
model.addReaction(['ES'], ['E', 'P'], 'kcat*ES', rxn_id='vcat')

<Reaction vcat>

Export model to SBML string

In [82]:
sbml_str = model.toSBML()
print(sbml_str)

<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model substanceUnits="mole" timeUnits="second" extentUnits="mole">
    <listOfUnitDefinitions>
      <unitDefinition id="per_second">
        <listOfUnits>
          <unit kind="second" exponent="-1" scale="0" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
    </listOfUnitDefinitions>
    <listOfCompartments>
      <compartment id="c1" spatialDimensions="3" size="1" units="litre" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="E" compartment="c1" initialAmount="5e-21" substanceUnits="mole" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="S" compartment="c1" initialAmount="1e-20" substanceUnits="mole" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="ES" compartment="c1" initialAmount="0" substanceUnits="mole" hasOnlySubstanc

Now, get the instructions to make such a model in simpleSBML

In [84]:
simpSBML_instructions = simplesbml.simplesbml.writeCodeFromString(sbml_str)
print(simpSBML_instructions)

import simplesbml
model = simplesbml.sbmlModel();
model.addSpecies(species_id='E', amt=5e-21);
model.addSpecies(species_id='S', amt=1e-20);
model.addSpecies(species_id='ES', amt=0.0);
model.addSpecies(species_id='P', amt=0.0);
model.addParameter(param_id='koff', val=0.2);
model.addParameter(param_id='kon', val=1000000.0);
model.addParameter(param_id='kcat', val=0.1);
model.addReaction(reactants=['E', 'S'], products=['ES'], expression='kon * E * S - koff * ES', rxn_id='veq');
model.addReaction(reactants=['ES'], products=['E', 'P'], expression='kcat * ES', rxn_id='vcat');


Now, get the antimony representation

In [87]:
import tellurium as te
sb_rep = te.antimonyConverter().sbmlToAntimony(moddd)[1]
print(sb_rep)

// Created by libAntimony v2.12.0
model *doc0()

  // Compartments and Species:
  compartment c1;
  species E in c1, S in c1, ES in c1, P in c1;

  // Reactions:
  veq: E + S => ES; kon*E*S - koff*ES;
  vcat: ES => E + P; kcat*ES;

  // Species initializations:
  E = 5e-21/c1;
  E has mole_per_litre;
  S = 1e-20/c1;
  S has mole_per_litre;
  ES = 0;
  ES has mole_per_litre;
  P = 0;
  P has mole_per_litre;

  // Compartment initializations:
  c1 = 1;
  c1 has litre;

  // Variable initializations:
  koff = 0.2;
  koff has per_second;
  kon = 1000000;
  kon has per_second;
  kcat = 0.1;
  kcat has per_second;

  // Other declarations:
  var koff, kon, kcat;
  const c1;

  // Unit definitions:
  unit per_second = 1 / second;
  unit substance = mole;
  unit extent = mole;
  unit time_unit = second;
  unit mole_per_litre = mole / litre;
end

